In [87]:
pip install nl2query azure-kusto-data 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [88]:
from nl2query import KustoQuery
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder

In [89]:
cluster_uri = "https://help.kusto.windows.net"

In [90]:
kcsb = KustoConnectionStringBuilder.with_interactive_login(cluster_uri)

In [92]:
with KustoClient(kcsb) as kusto_client:
    database = "ContosoSales"
    query = "Products | getschema"
    response = kusto_client.execute(database, query)

    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [113]:
with KustoClient(kcsb) as kusto_client:
    database = "FindMyPartner"
    query = "Partner | getschema"
    response = kusto_client.execute(database, query)

    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [114]:
cols = [x[0] for x in response.primary_results[0]]
print(cols)

['Partner', 'FocusGeography', 'PartnerType', 'Website', 'Contact', 'Logo', 'SuccessStories', 'Description', 'MicrosoftPartnerSite']


In [116]:
queryfier = KustoQuery(cols, 'Partner')


In [81]:
query = queryfier.generate_query('''show me the customers with a first name like "Ang" and a city like "Ball"''')

In [108]:
query = queryfier.generate_query('''show me all manufactures who make color name "Silver"''')

In [124]:
query = queryfier.generate_query('''show me all partner who has contact "info@dataCore.co.il"''')

In [125]:
query

'Partner | where Contact == "info@dataCore.co.il" | project Partner'

In [126]:
with KustoClient(kcsb) as kusto_client:
    database = "FindMyPartner"
    response = kusto_client.execute(database, query)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [123]:
print(response.primary_results[0])

{"name": "PrimaryResult", "kind": "PrimaryResult", "data": []}
